In [1]:
!pip install pydantic==1.10.9
!pip install openai
!pip install 'langchain[all]'     

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 1.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 8.1 MB/s eta 0:00:0000:0100:01
  Using cached openai-1.25.2-py3-none-any.whl.metadata (21 kB)
  Using cached anyio-4.3.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.0 MB/s eta 0:00:00
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached openai-1.25.2-py3-none-any.whl (312 kB)
Using cached anyio-4.3.0-py3-none-any.whl (85 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
Using cached httpcore-1.0.5-py3-none-any.whl (77 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (1

In [3]:
##OpenAI API Key 발급 받아서 사용해야됨
#import os

In [39]:
from langchain_core.prompts import PromptTemplate

my_template = '''{content} 라는 문장을 영어로 번역해줘'''
prompt = PromptTemplate(
    input_variables=['content'],
    template=my_template
)
prompt.format(content="서울 프롬프톤 대회를 나가기로 했엉")


'서울 프롬프톤 대회를 나가기로 했엉 라는 문장을 영어로 번역해줘'

In [41]:
from langchain_core.prompts import ChatMessagePromptTemplate

prompt = "{subject} 대회를 나가기로 했엉 라는 문장을 영어로 번역해줘"

chat_message_prompt = ChatMessagePromptTemplate.from_template(
    role="Jedi", template=prompt
)
chat_message_prompt.format(subject="서울 프롬프톤")

ChatMessage(content='서울 프롬프톤 대회를 나가기로 했엉 라는 문장을 영어로 번역해줘', role='Jedi')

In [45]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

human_prompt = "{word_count} 단어로 요약해줘."
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

chat_prompt = ChatPromptTemplate.from_messages(
    [MessagesPlaceholder(variable_name="conversation"), human_message_template]
)

In [46]:
from langchain_core.messages import AIMessage, HumanMessage

human_message = HumanMessage(content="대회에서 입상하는 방법")
ai_message = AIMessage(
    content="""\
1. 창의성

2. 실용성

3. 서류심사
"""
)

chat_prompt.format_prompt(
    conversation=[human_message, ai_message], word_count="11"
).to_messages()

[HumanMessage(content='대회에서 입상하는 방법'),
 AIMessage(content='1. 창의성\n\n2. 실용성\n\n3. 서류심사\n'),
 HumanMessage(content='11 단어로 요약해줘.')]

In [75]:
examples = [
    {"input": "안녕", "output": "2"},
    {"input": "코딩은 너무 어려워요", "output": "11"},
    {"input": "프로그래밍 할 때 중요한건 언어를 정하는 것", "output": "24"},
]

In [76]:
from langchain_core.example_selectors.base import BaseExampleSelector


class CustomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        new_word = input_variables["input"]
        new_word_length = len(new_word)
        best_match = None
        smallest_diff = float("inf")

        for example in self.examples:
            current_diff = abs(len(example["input"]) - new_word_length)

            if current_diff < smallest_diff:
                smallest_diff = current_diff
                best_match = example

        return [best_match]

In [77]:
example_selector = CustomExampleSelector(examples)

In [78]:
example_selector.select_examples({"input": "잘가라는 말에 의미"})

[{'input': '코딩은 너무 어려워요', 'output': '11'}]

In [79]:
example_selector.add_example({"input": "길이를 이용하여 프롬프트 챗봇을 만드는 방법 입니다.", "output": "29"})

In [84]:
example_selector.select_examples({"input": "길이를 이용한 챗봇은 길이를 통해 제어 할수 있는 문장에 사용됩니다."})

[{'input': '길이를 이용하여 프롬프트 챗봇을 만드는 방법 입니다.', 'output': '29'}]

In [85]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate.from_template("Input: {input} -> Output: {output}")

In [87]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Input: {input} -> Output:",
    prefix="기계번역에 많이 사용됩니다:",
    input_variables=["input"],
)

print(prompt.format(input="길이를 이용한 챗봇"))

기계번역에 많이 사용됩니다:

Input: 코딩은 너무 어려워요 -> Output: 11

Input: 길이를 이용한 챗봇 -> Output:


In [88]:
from langchain_core.example_selectors import LengthBasedExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=25,
)
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

In [89]:

print(dynamic_prompt.format(adjective="big"))

Give the antonym of every input

Input: happy
Output: sad

Input: tall
Output: short

Input: energetic
Output: lethargic

Input: sunny
Output: gloomy

Input: windy
Output: calm

Input: big
Output:


In [35]:
# An example with long input, so it selects only one example.
long_string = "big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else"
print(dynamic_prompt.format(adjective=long_string))

Give the antonym of every input

Input: happy
Output: sad

Input: big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else
Output:


In [36]:
# You can add an example to an example selector as well.
new_example = {"input": "big", "output": "small"}
dynamic_prompt.example_selector.add_example(new_example)
print(dynamic_prompt.format(adjective="enthusiastic"))

Give the antonym of every input

Input: happy
Output: sad

Input: tall
Output: short

Input: energetic
Output: lethargic

Input: sunny
Output: gloomy

Input: windy
Output: calm

Input: big
Output: small

Input: enthusiastic
Output:
